In [1]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.colheader_justify', 'left') 
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.width', None)  

In [2]:
file_path = '/home/carolus/Documents/school/green_ia/data/10_data/10_train_02.jsonl'
#file_path_bis = '/home/carolus/Documents/school/green_ia/data/17_data/17_openfoodfacts_02.jsonl'
num_samples = 500

In [3]:
def count_objects_in_jsonl(file_path):
    count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            count += 1
    return count
print(f"nombre objets {file_path}: {count_objects_in_jsonl(file_path)}")


nombre objets /home/carolus/Documents/school/green_ia/data/10_data/10_train_02.jsonl: 189340


In [4]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df
df = sample_jsonl_file(file_path, num_samples)

In [5]:
#occurrences = df['food_group'].value_counts()
#print(occurrences)

In [6]:
df.tail(1)

pnns_1 name                    ecoscore_tags  ecoscore_score  countries  \
499  4.0     pimienta negra molida  4.0            2.0             186.0       

    ecoscore_data                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [7]:
df.tail(100)

pnns_1 name                                                           \
400  6.0                                               knacks de volaille   
401  6.0                                                   saucisse sèche   
402  9.0                                                       fromy plus   
403  9.0                                                       cacahuètes   
404  7.0                                                  cranberry juice   
405  0.0                                          panettone al cioccolato   
406  1.0             coulis de tomate à l'huile d'olive vierge extra 1.1%   
407  2.0                                                  carottes râpées   
408  4.0                               mozzarella rallada especial pizzas   
409  6.0                        chiffonnade de jambon sec vpf de montagne   
410  3.0                           nature bio raisins secs sultanines bio   
411  4.0                                         grana padano dop riserva   
412  6.0                                    prosciutto cotto slta qualità   
413  0.0                                                       sour chews   
414  7.0                                                     bordeau rosé   
415  4.0                                leche desnatada calcio vitamina d   
416  7.0                                         pur jus de raisin muscat   
417  7.0                                                  sirop de citron   
418  4.0                                                  muenster cheese   
419  7.0                                  sirop fraise pur sucre de canne   
420  5.0                                        arroz branco são joão 2kg   
421  0.0                              miel d'acacia des vignes du pouilly   
422  5.0                                  pétales de maïs glacés au sucre   
423  4.0                                                              tej   
424  7.0                                               minute maid orange   
425  9.0                                                     rauchmandeln   
426  4.0                                                      dillspitzen   
427  4.0                                   original dutch cheese 8 slices   
428  4.0                                                             none   
429  1.0                         mayo mit senf, à la moutarde, con senape   
430  0.0                                                     occhi di bue   
431  6.0                             brochettes de boeuf haché nature x 4   
432  7.0                                       vin rouge côte chalonnaise   
433  6.0                              signature raw frozen salmon fillets   
434  0.0                            chocolate negro 70% con té y jengibre   
435  4.0                                            abondance au lait cru   
436  5.0                                                   aardappelpuree   
437  4.0                                                 bleu des causses   
438  0.0                                          orange fruit juice gels   
439  7.0                                               gaffels fassbrause   
440  6.0                                           saucisson sec pur porc   
441  6.0                            saucisson sec extra maigre à l'épaule   
442  1.0                                              sugo all’arrabbiata   
443  4.0                                               hähnchen schnitzel   
444  4.0                                          thé noir rêve d'automne   
445  2.0                                       purepower chilli con carne   
446  1.0                                                    ajvar ketchup   
447  1.0                                    moutarde forte au poivre vert   
448  3.0                                               fruit puree squizz   
449  0.0                         connaisseurs pâte d'amande chocolat noir   
450  4.0                                                  crème de marron   
4

In [8]:
col_nan = 'additives'
nb_empty = (df[col_nan].isna()).sum() 
#nb_empty = (df[col_nan] == 'empty').sum() 
nb_total = len(df)
pourcentage_empty = (nb_empty / nb_total) * 100
print(f"percetange empty in {col_nan}: {pourcentage_empty:.2f}%")

percetange empty in additives: 0.00%


In [9]:
#def load_jsonl_to_dataframe(file_path):
#    data = []
#    with open(file_path, 'r') as file:
#        for line in file:
#            data.append(json.loads(line))
#    df = pd.DataFrame(data)
#    return df
#
#def create_treemap_for_column(df, column_name):
#    if column_name not in df.columns:
#        print(f"warning, column: '{column_name}'does not exists")
#        return
#    data = df[column_name].value_counts().reset_index()
#    data.columns = [column_name, 'count']
#    fig = go.Figure(go.Treemap(
#        labels=data[column_name],
#        parents=[""] * len(data),
#        values=data['count'],
#        textinfo='label+value',  
#    ))
#    fig.update_layout(
#        title=f'treemap column: {column_name}',
#        paper_bgcolor='white',  
#        plot_bgcolor='white',   
#        font_color='black'      
#    )
#    fig.show()

In [10]:
#colonnes_a_travailler = ['stores']  
#df = load_jsonl_to_dataframe(file_path)
#for column in colonnes_a_travailler:
#    create_treemap_for_column(df, column)

In [11]:
#nb_empty = df['ecoscore_score'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in ecoscore_score before NaN reduction : {pourcentage_empty:.2f}%")

In [12]:
#nb_empty = df['countries'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in countries before NaN reduction : {pourcentage_empty:.2f}%")

In [13]:
#nb_empty = df['groups'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in groups before NaN reduction : {pourcentage_empty:.2f}%")

In [14]:
#nb_empty = df['text_data'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in text_data before NaN reduction : {pourcentage_empty:.2f}%")

In [15]:
#nb_empty = df['labels_note'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in labels_note before NaN reduction : {pourcentage_empty:.2f}%")

In [16]:
#nb_empty = df['categories'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in categories before NaN reduction : {pourcentage_empty:.2f}%")

In [17]:
#file_path_csv = '/home/carolus/Documents/school/green_ia/scripts/validation_predictions.csv'
#df_csv = pd.read_csv(file_path_csv)

In [18]:
#df_csv.head(60)

In [19]:
#df_csv['difference'] = df_csv['true'] - df_csv['predictions']
#max_diff = df_csv['difference'].max()
#min_diff = df_csv['difference'].min()
#mean_diff = df_csv['difference'].mean()  
#median_diff = df_csv['difference'].median()  
#total_rows = df_csv.shape[0]
#df_csv['difference'] = df_csv['true'] - df_csv['predictions']
#count_greater_than_2 = (df_csv['difference'].abs() > 2).sum()
#percentage = (count_greater_than_2 / total_rows) * 100

#print(f"Écart maximum: {max_diff}")
#print(f"Écart minimum: {min_diff}")
#print(f"Écart moyen: {mean_diff}")
#print(f"Écart médian: {median_diff}")
#print(f"poucentage lignes écart supérieur à 2% : {percentage}%")